In [1]:
import pandas as pd
import numpy as np
import time
import os
import glob
import json
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor

## Configuration

In [2]:
# Number of force cells in the robotic leg
N_CELLS = 8

# Path where the results are stored
RESULTS_PATH = '../../../results'
# ID of the training and test data resulting from this notebook, stored in RESULTS_PATH
DATA_ID = '0003_11042021'
# Number of folds in cross-validation
CV = 6

print('Model trained with data: ' + DATA_ID)

pd.set_option('display.max_columns', None)

Model trained with data: 0003_11042021


## Model

In [3]:
# Load data
X_train = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'X_train_{}.npy'.format(DATA_ID)))
X_test = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'X_test_{}.npy'.format(DATA_ID)))
Y_train = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'Y_train_{}.npy'.format(DATA_ID)))
Y_test = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'Y_test_{}.npy'.format(DATA_ID)))

params = {'max_depth': 15, 'max_features': 0.2, 'min_samples_leaf': 0.01, 'min_samples_split': 0.001, 'n_estimators': 100}

results_ls = []

In [4]:
# Setup the model with the best parameters
model = RandomForestRegressor(**params, random_state=0, n_jobs=-1, verbose=0)

t_start = time.time()
model.fit(X_train, Y_train)
tr_time = time.time() - t_start
print('Training time: {:.4f}'.format(tr_time))

train_preds = model.predict(X_train)
test_preds = model.predict(X_test)

results = {
    'Train': {
        'MAE': mean_absolute_error(Y_train, train_preds, multioutput='raw_values'),
        'MSE': mean_squared_error(Y_train, train_preds, multioutput='raw_values'),
        'R2': r2_score(Y_train, train_preds, multioutput='raw_values')
    },
    'Test': {
        'MAE': mean_absolute_error(Y_test, test_preds, multioutput='raw_values'),
        'MSE': mean_squared_error(Y_test, test_preds, multioutput='raw_values'),
        'R2': r2_score(Y_test, test_preds, multioutput='raw_values')
    }       
    
}

results_summary = {}
results_summary['Training time'] = tr_time
# Display the score mean and standard deviation of each axis
for subset in ['Train', 'Test']:
    for f, force in enumerate(['Fx', 'Fy', 'Fz']):
        for loss in ['MAE', 'MSE', 'R2']:
            scores = [results[subset][loss][i + f] for i in range(0, N_CELLS * 3, 3)]
            print(' '.join([subset, force, loss]) + ': {:.4f} ± {:.4f}'.format(np.mean(scores), np.std(scores)))
            results_summary[' '.join([subset, force, loss])] = '{:.4f} ± {:.4f}'.format(np.mean(scores), np.std(scores))
            
results_ls.append(results_summary)

Training time: 6.6160
Train Fx MAE: 8.6461 ± 2.4048
Train Fx MSE: 151.6214 ± 77.8937
Train Fx R2: 0.5655 ± 0.0612
Train Fy MAE: 8.7525 ± 6.0748
Train Fy MSE: 238.7282 ± 265.7849
Train Fy R2: 0.4389 ± 0.1117
Train Fz MAE: 11.2891 ± 3.9985
Train Fz MSE: 300.2048 ± 246.7115
Train Fz R2: 0.5458 ± 0.0617
Test Fx MAE: 13.0032 ± 5.0952
Test Fx MSE: 311.5999 ± 249.7830
Test Fx R2: 0.3473 ± 0.2863
Test Fy MAE: 10.7406 ± 7.7562
Test Fy MSE: 378.7206 ± 481.6318
Test Fy R2: 0.3404 ± 0.1027
Test Fz MAE: 19.0024 ± 6.5255
Test Fz MSE: 687.2684 ± 593.5296
Test Fz R2: 0.3611 ± 0.2479


In [5]:
results = {
    'Train': {
        'MAE': [],
        'MSE': [],
        'R2': []
    },
    'Test': {
        'MAE': [],
        'MSE': [],
        'R2': []
    }       
    
}

tr_time = []
for target in range(Y_train.shape[1]):

    # Setup the model with the best parameters
    model = RandomForestRegressor(**params, random_state=0, n_jobs=-1, verbose=0)

    t_start = time.time()
    model.fit(X_train, Y_train[:, target])
    tr_time.append(time.time() - t_start)
    
    train_preds = model.predict(X_train)
    test_preds = model.predict(X_test)

    results['Train']['MAE'].append(mean_absolute_error(Y_train[:, target], train_preds))
    results['Train']['MSE'].append(mean_squared_error(Y_train[:, target], train_preds))
    results['Train']['R2'].append(r2_score(Y_train[:, target], train_preds))
    results['Test']['MAE'].append(mean_absolute_error(Y_test[:, target], test_preds))
    results['Test']['MSE'].append(mean_squared_error(Y_test[:, target], test_preds))
    results['Test']['R2'].append(r2_score(Y_test[:, target], test_preds))
    
print('Training time: {:.4f}'.format(sum(tr_time)))

results_summary = {}
results_summary['Training time'] = sum(tr_time)
# Display the score mean and standard deviation of each axis
for subset in ['Train', 'Test']:
    for f, force in enumerate(['Fx', 'Fy', 'Fz']):
        for loss in ['MAE', 'MSE', 'R2']:
            scores = [results[subset][loss][i + f] for i in range(0, N_CELLS * 3, 3)]
            print(' '.join([subset, force, loss]) + ': {:.4f} ± {:.4f}'.format(np.mean(scores), np.std(scores)))
            results_summary[' '.join([subset, force, loss])] = '{:.4f} ± {:.4f}'.format(np.mean(scores), np.std(scores))
            
results_ls.append(results_summary)

Training time: 97.3379
Train Fx MAE: 8.2300 ± 2.2604
Train Fx MSE: 139.6984 ± 69.3447
Train Fx R2: 0.5994 ± 0.0514
Train Fy MAE: 8.3189 ± 5.8516
Train Fy MSE: 226.1526 ± 260.1663
Train Fy R2: 0.4990 ± 0.0828
Train Fz MAE: 10.7197 ± 3.9589
Train Fz MSE: 277.0319 ± 250.6741
Train Fz R2: 0.5921 ± 0.0460
Test Fx MAE: 12.8970 ± 5.2116
Test Fx MSE: 310.9195 ± 253.9345
Test Fx R2: 0.3544 ± 0.2769
Test Fy MAE: 10.5866 ± 7.7704
Test Fy MSE: 359.1706 ± 461.7569
Test Fy R2: 0.3808 ± 0.1270
Test Fz MAE: 18.4798 ± 6.8681
Test Fz MSE: 675.8267 ± 645.0185
Test Fz R2: 0.4042 ± 0.1982


In [6]:
pd.DataFrame(results_ls)

,Training time,Train Fx MAE,Train Fx MSE,Train Fx R2,Train Fy MAE,Train Fy MSE,Train Fy R2,Train Fz MAE,Train Fz MSE,Train Fz R2,Test Fx MAE,Test Fx MSE,Test Fx R2,Test Fy MAE,Test Fy MSE,Test Fy R2,Test Fz MAE,Test Fz MSE,Test Fz R2
0,6.616011,8.6461 ± 2.4048,151.6214 ± 77.8937,0.5655 ± 0.0612,8.7525 ± 6.0748,238.7282 ± 265.7849,0.4389 ± 0.1117,11.2891 ± 3.9985,300.2048 ± 246.7115,0.5458 ± 0.0617,13.0032 ± 5.0952,311.5999 ± 249.7830,0.3473 ± 0.2863,10.7406 ± 7.7562,378.7206 ± 481.6318,0.3404 ± 0.1027,19.0024 ± 6.5255,687.2684 ± 593.5296,0.3611 ± 0.2479
1,97.337948,8.2300 ± 2.2604,139.6984 ± 69.3447,0.5994 ± 0.0514,8.3189 ± 5.8516,226.1526 ± 260.1663,0.4990 ± 0.0828,10.7197 ± 3.9589,277.0319 ± 250.6741,0.5921 ± 0.0460,12.8970 ± 5.2116,310.9195 ± 253.9345,0.3544 ± 0.2769,10.5866 ± 7.7704,359.1706 ± 461.7569,0.3808 ± 0.1270,18.4798 ± 6.8681,675.8267 ± 645.0185,0.4042 ± 0.1982
